In [1]:
files_folder = 'Data'
Results_folder = 'Results'

## Split header and data

In [2]:
import csv
import chardet
from os import listdir, path, makedirs

In [3]:
def files_path_searcher(files_folder):
    files = [path.join(files_folder, f) for f in listdir(files_folder) if path.isfile(path.join(files_folder, f))]
    return files

In [4]:
def path_helper(file_path):
    file_name = path.split(file_path)[1].split(sep='.')[0]
    folder_path = path.join(Results_folder, file_name)
    header_path = path.join(folder_path,'header.txt')
    data_path = path.join(folder_path,'data.csv')
    fig_path = path.join(folder_path,file_name)
    if not path.exists(folder_path):
        makedirs(folder_path)
    return [header_path, data_path,fig_path,folder_path]

In [5]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    return chardet.detect(raw_data)['encoding']

def split_file(input_file, txt_output, csv_output):
    encoding = detect_encoding(input_file)
    
    with open(input_file, 'r', encoding=encoding) as infile, \
         open(txt_output, 'w', encoding='utf-8') as txt_out, \
         open(csv_output, 'w', newline='', encoding='utf-8') as csv_out:
        
        csv_writer = csv.writer(csv_out)
        flag = 0
        for line in infile:
            if flag == 1:
                # Write this line and all subsequent lines to the CSV file
                csv_writer.writerow(line.strip().split('\t'))
                for line in infile:
                    csv_writer.writerow(line.strip().split('\t'))
            else:
                # Write lines before the specified line to the TXT file
                txt_out.write(line)
            if 'rel. 1/cm\tCCD cts' in line:
                flag = 1

## Read data

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from lmfit import Model, Parameters
from scipy.signal import find_peaks

In [7]:
def read_data(data_path, fig_path):
    data = np.genfromtxt(data_path, delimiter=',',)
    x = data[:,0]
    y = data[:,1]
    plt.figure(1)
    plt.xlabel('Raman Shift (1/cm)')
    plt.ylabel('Intensity (a.u)')
    plt.plot(x,y)
    plt.savefig(fig_path+ '.svg', dpi=300, bbox_inches = 'tight')
    plt.savefig(fig_path+ '.png', dpi=300, bbox_inches = 'tight')
    plt.close(1)
    return [x,y]

## Analyze data

In [8]:
def lorentzian(x, amplitude, center, sigma):
    return amplitude * (sigma**2 / ((x - center)**2 + sigma**2))

def analyze_data(data, fig_path, folder_path):
    x, y = data[0], data[1]
    # Find peaks
    peaks, _ = find_peaks(y, height=0.5, distance=50)
    
    # Create a composite model with multiple Lorentzian peaks
    model = None
    params = Parameters()
    for i, peak in enumerate(peaks):
        peak_model = Model(lorentzian, prefix=f'p{i}_')
        if model is None:
            model = peak_model
        else:
            model += peak_model
        params.add(f'p{i}_amplitude', value=y[peak], min=0)
        params.add(f'p{i}_center', value=x[peak], min=x.min(), max=x.max())
        params.add(f'p{i}_sigma', value=20, min=1)
    
    # Fit the model to the data
    result = model.fit(y, params, x=x)
    
    # Print the fit results
    fit_results_path = path.join(folder_path,'results_fitted.txt')
    with open(fit_results_path, 'w', encoding='utf-8') as txt_out:
        txt_out.write(result.fit_report())
    
    # Plot the results
    plt.figure(2,figsize=(12, 6))
    plt.plot(x, y, 'b-', label='Data')
    plt.plot(x, result.best_fit, 'r-', label='Fit')
    plt.plot(x[peaks], y[peaks], "ko", label='Detected Peaks')
    
    # Plot individual Lorentzian components
    components = result.eval_components(x=x)
    for i, component in enumerate(components):
        plt.plot(x, components[component], '--', label=f'Peak {i+1}')
    
    plt.legend(bbox_to_anchor=(0, 1.08, 1, 0.2), loc="lower left", mode="expand", borderaxespad=0, ncol=6)
    plt.xlabel('Raman Shift (cm^-1)')
    plt.ylabel('Intensity')
    plt.title('RAMAN Data Analysis with Multiple Lorentzian Models')
    plt.plot()
    plt.savefig(fig_path+ '_fitted.svg', dpi=300, bbox_inches = 'tight')
    plt.savefig(fig_path+ '_fitted.png', dpi=300, bbox_inches = 'tight')
    plt.close(2)
    
    # Extract and print fitted parameters
    peaks_results_path = path.join(folder_path,'results_peaks_info.txt')
    with open(peaks_results_path, 'w', encoding='utf-8') as txt_out:
        for i in range(len(peaks)):
            amp = result.params[f'p{i}_amplitude'].value
            cen = result.params[f'p{i}_center'].value
            sig = result.params[f'p{i}_sigma'].value
            txt_out.write(f"Peak {i+1}:\n")
            txt_out.write(f"  Amplitude: {amp:.2f}\n")
            txt_out.write(f"  Center: {cen:.2f}\n")
            txt_out.write(f"  Sigma: {sig:.2f}\n")

## Execute

In [9]:
file_paths = files_path_searcher(files_folder)
for file_path in file_paths:
    data = []
    header_path, data_path, fig_path, results_path = path_helper(file_path)
    split_file(file_path, header_path, data_path)
    data = read_data(data_path,fig_path)
    analyze_data(data, fig_path, results_path)